## packages

In [1]:
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import inflect
from textblob import TextBlob
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer #该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
from sklearn.feature_extraction.text import TfidfTransformer

## define stopwords
sr = stopwords.words('english')
p = inflect.engine()
wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

Using TensorFlow backend.


## define function "read_json" and "write_json"

In [2]:
def read_json(path):
    with open(path, 'r',encoding='utf-8') as f:
        iter_f = iter(f)
        line = f.readline()
        text = []
        for line in iter_f: 
            d = json.loads(line)
            print(d)
            text.append(d)
        f.close()
    return(text)
    
def write_json(path, data):
    with open(path, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    return 0

## read "seafood_review.json"

In [3]:
# seafood_review=read_json("example.json")
# with open("example.json"，encoding='utf-8') as file:
#     result = json.loads(file)
# file.close()
# seafood_review=json.loads("example.json")
with open('example.json', 'r',encoding='utf-8') as f:
    data = f.read()
    json_data = json.loads(data)
f.close()

In [4]:
type(json_data)

list

In [5]:
json_data[1]

{'review_id': '8NBE5q5UWLQUqWyM7GpQ2g',
 'user_id': 'StZTVDuFzahNvjl5qu6l7Q',
 'business_id': 'nsNONDHbV7Vudqh21uicqw',
 'stars': 2.0,
 'useful': 1,
 'funny': 1,
 'cool': 1,
 'text': 'Seems old and tired!   I ate here frequently about 20 years ago and returned for happy hour this year.  This used to be a good place many years ago, but too many great restaurants have opened in recent years to eat here.  Strange rules about what you can eat in the bar area.  Food just meh.',
 'date': '2018-05-08 17:18:20'}

In [6]:
len(json_data)

543

In [24]:
## review processiong
review_train_clean = [[1]]*len(data_train)
##a list  with dict element,you can get a summary of frequency of one review by [i] and frequency for a specific word by [i]['word']
frequency_train_clean = [[1]]*len(data_train)  
for i in tqdm(range(len(data_train))):
    #Change n't into not
    x = re.sub(r'n\'t',' not',data_train.iloc[i]['text'])
    #Change not adj into not_adj
    x = re.sub(r'not ','not_',x)
    #Split into words
    x = word_tokenize(x)
    #Remove punctuation
    x = [w.translate(table) if not re.match(r'not_.*', w) else w for w in x]
    #Change numbers into words
    x = [p.number_to_words(w) if w.isdigit() else w  for w in x ]
    #Remove non alphabetic
    x = [w for w in x if w.isalpha() or re.match(r'not_.*',w)]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    review_train_clean[i] = x
    frequency_train_clean[i] = collections.Counter(x)

[nltk_data] Downloading package stopwords to C:\Users\GAO
[nltk_data]     ZEJIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [8]:
# nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\GAO
[nltk_data]     ZEJIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [ ]:
# nltk.download()

In [10]:
## example    
#     x = "Worst Gyms I met. And I wouldn't come again 3. not great."
    x = "Seems old and tired!   I ate here frequently about 20 years ago and returned for happy hour this year.  This used to be a good place many years ago, but too many great restaurants have opened in recent years to eat here.  Strange rules about what you can eat in the bar area.  Food just meh."
    p = inflect.engine()
    x = re.sub(r'n\'t',' not',x)
    #Change not adj into not_adj
    x = re.sub(r'not ','not_',x)
    #Split into words
    x = word_tokenize(x)
    #Remove punctuation
    x = [w.translate(table) if not re.match(r'not_.*', w) else w for w in x]
    print(x)
    #Change numbers into words
    x = [p.number_to_words(w) if w.isdigit() else w  for w in x ]
    print(x)
    #Remove not alphabetic
    x = [w for w in x if w.isalpha() or re.match(r'not_.*',w)]
    #Convert to lower case
    x = [w.lower() for w in x]
    print(x)
    #Remove stop words
    x = [w for w in x if not w in sr]
    print(x)
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    print(x)

['Seems', 'old', 'and', 'tired', '', 'I', 'ate', 'here', 'frequently', 'about', '20', 'years', 'ago', 'and', 'returned', 'for', 'happy', 'hour', 'this', 'year', '', 'This', 'used', 'to', 'be', 'a', 'good', 'place', 'many', 'years', 'ago', '', 'but', 'too', 'many', 'great', 'restaurants', 'have', 'opened', 'in', 'recent', 'years', 'to', 'eat', 'here', '', 'Strange', 'rules', 'about', 'what', 'you', 'can', 'eat', 'in', 'the', 'bar', 'area', '', 'Food', 'just', 'meh', '']
['Seems', 'old', 'and', 'tired', '', 'I', 'ate', 'here', 'frequently', 'about', 'twenty', 'years', 'ago', 'and', 'returned', 'for', 'happy', 'hour', 'this', 'year', '', 'This', 'used', 'to', 'be', 'a', 'good', 'place', 'many', 'years', 'ago', '', 'but', 'too', 'many', 'great', 'restaurants', 'have', 'opened', 'in', 'recent', 'years', 'to', 'eat', 'here', '', 'Strange', 'rules', 'about', 'what', 'you', 'can', 'eat', 'in', 'the', 'bar', 'area', '', 'Food', 'just', 'meh', '']
['seems', 'old', 'and', 'tired', 'i', 'ate', 'he